In [53]:
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error 
from keras.callbacks import ModelCheckpoint

In [49]:
df=read_csv("Delays_W_L_FAMIRRORSimulated_50K.csv", sep="\t")
dataset = df.values

In [50]:
y_val= df[df.columns[56]]
x_data = df[df.columns[:56]]
X_train, X_eval,y_train,y_eval=train_test_split(x_data,y_val,test_size=0.2,random_state=101)

In [65]:
X_train.shape

(40000, 56)

In [35]:
# def larger_model():
#     model = Sequential()
#     model.add(Dense(28, input_dim=56, kernel_initializer='normal', activation='relu'))
# #     model.add(Dense(56, kernel_initializer='normal', activation='relu'))
#     model.add(Dense(1, kernel_initializer='normal'))
#     # Compile model
#     model.compile(loss='mean_squared_error', optimizer='adam')
#     return model

NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(56, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(10, kernel_initializer='normal',activation='relu'))
# NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
# NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 56)                3192      
_________________________________________________________________
dense_8 (Dense)              (None, 10)                570       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 11        
Total params: 3,773
Trainable params: 3,773
Non-trainable params: 0
_________________________________________________________________


In [36]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [37]:
NN_model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)

Train on 32000 samples, validate on 8000 samples
Epoch 1/100
32000/32000 [==============================] - 1s 31us/step - loss: 1.1685e-04 - mean_absolute_error: 1.1685e-04 - val_loss: 2.3302e-04 - val_mean_absolute_error: 2.3302e-04

Epoch 00001: val_loss improved from inf to 0.00023, saving model to Weights-001--0.00023.hdf5
Epoch 2/100
32000/32000 [==============================] - 1s 31us/step - loss: 1.0784e-04 - mean_absolute_error: 1.0784e-04 - val_loss: 1.0682e-04 - val_mean_absolute_error: 1.0682e-04

Epoch 00002: val_loss improved from 0.00023 to 0.00011, saving model to Weights-002--0.00011.hdf5
Epoch 3/100
32000/32000 [==============================] - 1s 37us/step - loss: 1.0864e-04 - mean_absolute_error: 1.0864e-04 - val_loss: 7.7302e-06 - val_mean_absolute_error: 7.7302e-06

Epoch 00003: val_loss improved from 0.00011 to 0.00001, saving model to Weights-003--0.00001.hdf5
Epoch 4/100
32000/32000 [==============================] - 1s 32us/step - loss: 1.0863e-04 - mean_ab


Epoch 00034: val_loss did not improve from 0.00001
Epoch 35/100
32000/32000 [==============================] - 1s 28us/step - loss: 1.0864e-04 - mean_absolute_error: 1.0864e-04 - val_loss: 3.1236e-05 - val_mean_absolute_error: 3.1236e-05

Epoch 00035: val_loss did not improve from 0.00001
Epoch 36/100
32000/32000 [==============================] - 1s 30us/step - loss: 1.0823e-04 - mean_absolute_error: 1.0823e-04 - val_loss: 2.8741e-04 - val_mean_absolute_error: 2.8741e-04

Epoch 00036: val_loss did not improve from 0.00001
Epoch 37/100
32000/32000 [==============================] - 1s 28us/step - loss: 1.0782e-04 - mean_absolute_error: 1.0782e-04 - val_loss: 2.3313e-04 - val_mean_absolute_error: 2.3313e-04

Epoch 00037: val_loss did not improve from 0.00001
Epoch 38/100
32000/32000 [==============================] - 1s 29us/step - loss: 1.0784e-04 - mean_absolute_error: 1.0784e-04 - val_loss: 1.0704e-04 - val_mean_absolute_error: 1.0704e-04

Epoch 00038: val_loss did not improve from 

In [45]:
wights_file = 'Weights-003--0.00001.hdf5' # choose the best checkpoint 
NN_model.load_weights(wights_file) # load it
print(NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error']))

None


In [42]:
predictions = NN_model.predict(X_eval)

In [43]:
print(y_eval)

18447    4.943910e-11
17768    2.204896e-11
45879    2.489709e-11
16972    2.774900e-11
12165    1.083044e-10
             ...     
40899    2.404310e-11
7806     2.939346e-11
41481    3.239289e-11
21802    3.542513e-11
42453    1.797644e-11
Name: delay_lh_nodeaco, Length: 10000, dtype: float64


In [44]:
for i in range(1000):
    print(predictions[i], y_eval.iloc[i])

[-7.730116e-06] 4.9439102269999996e-11
[-7.730116e-06] 2.2048957740000002e-11
[-7.730116e-06] 2.489708941e-11
[-7.730116e-06] 2.774900333e-11
[-7.730116e-06] 1.08304372e-10
[-7.730116e-06] 3.611094356e-11
[-7.730116e-06] 2.459907273e-11
[-7.730116e-06] 3.655773953e-11
[-7.730116e-06] 4.1953943819999996e-11
[-7.730116e-06] 1.4356442819999998e-10
[-7.730116e-06] 2.716494561e-11
[-7.730116e-06] 3.35001091e-11
[-7.730116e-06] 5.962758013e-11
[-7.730116e-06] 3.942331229e-11
[-7.730116e-06] 5.594167912e-11
[-7.730116e-06] 6.236387056e-11
[-7.730116e-06] 5.323298824e-11
[-7.730116e-06] 9.661160836e-11
[-7.730116e-06] 7.18285061e-11
[-7.730116e-06] 4.8953177960000003e-11
[-7.730116e-06] 2.159124863e-11
[-7.730116e-06] 1.5843460680000003e-10
[-7.730116e-06] 8.934609687e-11
[-7.730116e-06] 4.1938404229999995e-11
[-7.730116e-06] 3.464559677e-11
[-7.730116e-06] 1.2424656049999999e-10
[-7.730116e-06] 3.3586111359999996e-11
[-7.730116e-06] 6.110914941e-11
[-7.730116e-06] 2.177782661e-11
[-7.730116e-

In [51]:
train_X, val_X, train_y, val_y = train_test_split(x_data, y_val, test_size = 0.25, random_state = 14)

In [54]:
model = RandomForestRegressor()
model.fit(train_X,train_y)

# Get the mean absolute error on the validation data
predicted_prices = model.predict(val_X)
MAE = mean_absolute_error(val_y , predicted_prices)
print('Random forest validation MAE = ', MAE)

Random forest validation MAE =  2.24598323205539e-11


In [58]:
predicted_prices = model.predict(val_X)
predicted_prices

array([5.15909197e-11, 5.15909197e-11, 5.15909197e-11, ...,
       5.15909197e-11, 5.15909197e-11, 5.15909197e-11])

In [57]:
for i in range(1000):
    print(predicted_prices[i], y_eval.iloc[i])

5.159091970611288e-11 4.9439102269999996e-11
5.159091970611288e-11 2.2048957740000002e-11
5.159091970611288e-11 2.489708941e-11
5.159091970611288e-11 2.774900333e-11
5.159091970611288e-11 1.08304372e-10
5.159091970611288e-11 3.611094356e-11
5.159091970611288e-11 2.459907273e-11
5.159091970611288e-11 3.655773953e-11
5.159091970611288e-11 4.1953943819999996e-11
5.159091970611288e-11 1.4356442819999998e-10
5.159091970611288e-11 2.716494561e-11
5.159091970611288e-11 3.35001091e-11
5.159091970611288e-11 5.962758013e-11
5.159091970611288e-11 3.942331229e-11
5.159091970611288e-11 5.594167912e-11
5.159091970611288e-11 6.236387056e-11
5.159091970611288e-11 5.323298824e-11
5.159091970611288e-11 9.661160836e-11
5.159091970611288e-11 7.18285061e-11
5.159091970611288e-11 4.8953177960000003e-11
5.159091970611288e-11 2.159124863e-11
5.159091970611288e-11 1.5843460680000003e-10
5.159091970611288e-11 8.934609687e-11
5.159091970611288e-11 4.1938404229999995e-11
5.159091970611288e-11 3.464559677e-11
5.15